<a href="https://colab.research.google.com/github/selgebali/Colabs/blob/main/RTG_RelationType_RTGs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resource Network Visualization Script

## Overview
This Python script fetches data from the DataCite API to build and visualize resource relationships as interactive network graphs. The script highlights connections between resources, their types, and relationships using Plotly and NetworkX. The results are saved in an HTML file for easy sharing and exploration.

## Features
- **Fetch Data:** Dynamically query the DataCite API with specific relationship and resource types.
- **Parse Relationships:** Extract and group related items based on specified criteria.
- **Visualize Networks:** Create interactive network graphs using Plotly and NetworkX.
- **Save Results:** Append all visualizations to a single HTML file.

## Prerequisites
Ensure the following libraries are installed before running the script:

- `requests`: For making API requests.
- `json`: To parse API responses.
- `pandas`: For potential data manipulation.
- `networkx`: To create and manage network graphs.
- `plotly`: For interactive graph visualization.

Install the required libraries using pip:
```bash
pip install requests pandas networkx plotly
```

## Script Details

### Key Functions

#### 1. `fetch_api_data(url, relationship_type, related_item_type=None)`
Fetches data from the DataCite API and extracts resources based on the specified relationship type and optional related item type.
- **Inputs:**
  - `url`: The API endpoint.
  - `relationship_type`: The type of relationship to filter (e.g., `"IsCollectedBy,Collects"`).
  - `related_item_type`: Optional filter for related item types (e.g., `"Instrument"`).
- **Output:** A list of parsed resource dictionaries.
- **Logs:** API URL, response status, and parsed resource details.

#### 2. `parse_resources(data, relationship_type, related_item_type=None)`
Parses resource data and extracts relationships based on the specified criteria.
- **Inputs:**
  - `data`: The data section of the API response.
  - `relationship_type`: The relationship type to filter.
  - `related_item_type`: Optional filter for related item types.
- **Output:** A list of dictionaries containing resource and relationship details.
- **Logs:** Parsed resources for debugging.

#### 3. `visualize_resource_network(resources, html_file)`
Visualizes resource relationships as network graphs and appends them to an HTML file.
- **Inputs:**
  - `resources`: Parsed resource data with relationships.
  - `html_file`: File to save the visualizations.
- **Output:** Network graphs displayed in the browser and saved to the HTML file.
- **Logs:** Graph generation and HTML file updates.

### Example Usage

#### Parameters
- **Relationship Type:** The type of relationship to visualize (e.g., `"IsCollectedBy,Collects"`).
- **Related Item Type:** The resource type to filter (e.g., `"Instrument"`).
- **API URL:**
```python
api_url = "https://api.datacite.org/dois?query=relatedIdentifiers.relationType:IsCollectedBy%20OR%20relatedIdentifiers.relationType:Collects"
```
- **HTML File:** The output file name (e.g., `"related_resource_network_graph.html"`).

#### Steps
1. Initialize an HTML file to store the visualizations.
2. Fetch and parse resource data from the API.
3. Visualize relationships as network graphs.
4. Append all graphs to the HTML file and close it.

### Execution
```python
relationship_type = "IsCollectedBy,Collects"
related_item_type = "Instrument"
api_url = "https://api.datacite.org/dois?query=relatedIdentifiers.relationType:IsCollectedBy%20OR%20relatedIdentifiers.relationType:Collects"
html_file = "related_resource_network_graph.html"

# Initialize HTML file
with open(html_file, 'w') as f:
    f.write("<html><head><title>Related Resource Network Graphs</title></head><body>")

# Fetch and visualize data
resources = fetch_api_data(api_url, relationship_type, related_item_type)
visualize_resource_network(resources, html_file)

# Finalize the HTML file
with open(html_file, 'a') as f:
    f.write("</body></html>")
```

### Output
- **Network Graphs:** Interactive visualizations for each group of related items.
- **HTML File:** All graphs are saved in `related_resource_network_graph.html`.

## Customization
- **Graph Style:** Adjust `custom_colors` or modify the `visualize_resource_network` function to customize node and edge styles.
- **Filter Criteria:** Modify `relationship_type` and `related_item_type` for different API queries.

## Debugging
The script includes debug logs to trace API calls, parsed data, and visualization progress.

## Limitations
- The script assumes the DataCite API's structure is consistent. Changes to the API may require updates to parsing logic.

## Contributing
Contributions are welcome! Please submit issues or pull requests to enhance functionality or address bugs.

In [1]:
import requests
import json
import pandas as pd
import networkx as nx
import plotly.graph_objs as go
import plotly.io as pio

# Custom color palette
custom_colors = ['#243B54', '#00B1E2', '#5B88B9', '#46BCAB', '#90D7CD', '#BC2B66']

# Function to fetch data from the API endpoint with dynamic resource type and relationship type
def fetch_api_data(url, relationship_type, related_item_type=None):
    resources = []
    while url:
        response = requests.get(url)  # Send a request to the given API URL
        print(f"Fetching data from URL: {url}")  # Debug log: API URL being fetched
        response.raise_for_status()  # Raise an error if the request was unsuccessful
        data = response.json()  # Parse the JSON response
        print(f"API response status code: {response.status_code}")  # Debug log: Status code of response
        resources.extend(parse_resources(data['data'], relationship_type, related_item_type))  # Parse resources and add to list
        url = data['links'].get('next')  # Update the URL to the next page, if any
    return resources

# Function to parse resources and their relationships
def parse_resources(data, relationship_type, related_item_type=None):
    resources = []
    for item in data:
        # Normalize resourceTypeGeneral to handle variations like hyphens and casing
        resource_type_general = item['attributes']['types'].get('resourceTypeGeneral', '').replace('-', '').lower()
        resource_data = {
            "doi": item['attributes'].get('doi', 'No DOI available'),
            "resourceTypeGeneral": item['attributes']['types'].get('resourceTypeGeneral', 'No resourceTypeGeneral available'),
            "creators": [],
            "contributors": [],
            "relatedItems": []
        }
        # Add related items based on relationship type and related item type
        if 'relatedIdentifiers' in item['attributes']:
            for related in item['attributes']['relatedIdentifiers']:
                relation_type = related.get('relationType', '')
                resource_type_related = related.get('resourceTypeGeneral', '')
                if relation_type and relationship_type and relation_type.lower() in relationship_type.lower():
                    if related_item_type is None or (resource_type_related and resource_type_related.replace('-', '').lower() == related_item_type.replace('-', '').lower()):
                        resource_data['relatedItems'].append({
                            "identifier": related.get('relatedIdentifier', 'No identifier available'),
                            "relationType": relation_type,
                            "resourceTypeGeneral": resource_type_related
                        })
        resources.append(resource_data)  # Append parsed resource data to the list
    print(f"Parsed resources: {resources}")  # Debug log: Parsed resources
    return resources

# Function to visualize the resource network graph
def visualize_resource_network(resources, html_file):
    grouped_resources = {}  # Group resources by related identifier

    # Group resources by related identifier
    for resource in resources:
        for related_item in resource["relatedItems"]:
            related_identifier = related_item["identifier"]
            if related_identifier not in grouped_resources:
                grouped_resources[related_identifier] = []
            grouped_resources[related_identifier].append((resource, related_item))

    # Create separate graphs for each group of related items
    for related_identifier, group in grouped_resources.items():
        G = nx.DiGraph()  # Create a directed graph for each group

        for resource, related_item in group:
            # Add the central resource node
            resource_node_label = f"{resource['resourceTypeGeneral']}: DOI: {resource['doi']}"
            G.add_node(resource_node_label, label=resource_node_label, size=300, color=custom_colors[0])

            # Add the related item node
            related_label = f"{related_item['resourceTypeGeneral']}: {related_item['identifier']}"
            G.add_node(related_label, label=related_label, size=300, color=custom_colors[1])
            G.add_edge(resource_node_label, related_label, relationType=related_item["relationType"])

        # Generate positions for the nodes
        pos = nx.spring_layout(G, seed=42)

        node_x, node_y, node_text, node_hovertext, node_sizes, node_colors = [], [], [], [], [], []
        for node in G.nodes(data=True):
            x, y = pos[node[0]]
            node_x.append(x)
            node_y.append(y)
            node_text.append(node[1]['label'])
            node_hovertext.append(node[1]['label'])
            node_sizes.append(node[1]['size'])
            node_colors.append(node[1]['color'])

        # Prepare edge attributes
        edge_x, edge_y = [], []
        for edge in G.edges(data=True):
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])

        # Create edge trace
        edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=3, color='gray'), hoverinfo='none', mode='lines')

        # Create node trace
        node_trace = go.Scatter(
            x=node_x, y=node_y, text=node_text, hovertext=node_hovertext, mode='markers+text',
            textposition='middle center', marker=dict(size=node_sizes, color=node_colors)
        )

        # Create the Plotly figure
        fig = go.Figure(data=[edge_trace, node_trace],
                        layout=go.Layout(
                            title=f"Network Graph of Resources and Relationships for {related_identifier}",
                            titlefont_size=16, showlegend=False, hovermode='closest', margin=dict(b=20, l=5, r=5, t=40),
                            width=1500, height=1000, xaxis=dict(showgrid=False, zeroline=False, showticklabels=False), yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                            plot_bgcolor='white',  # Set plot background color here
                            paper_bgcolor='white'  # Set overall figure background color
                        )
                      )
        fig.show()
        print(f"Displayed network graph for related identifier: {related_identifier}")  # Debug log: Network graph displayed

        # Append to HTML file
        with open(html_file, 'a') as f:
            f.write(pio.to_html(fig, include_plotlyjs='cdn'))
            print(f"Appended network graph to HTML file: {html_file}")  # Debug log: Graph appended to HTML

# Example usage
relationship_type = "IsCollectedBy,Collects"
related_item_type = "Instrument"
api_url = f"https://api.datacite.org/dois?query=relatedIdentifiers.relationType:IsCollectedBy%20OR%20relatedIdentifiers.relationType:Collects"

html_file = "related_resource_network_graph.html"

# Initialize HTML file with basic structure
with open(html_file, 'w') as f:
    f.write("<html><head><title>Related Resource Network Graphs</title></head><body>")
    print(f"Initialized HTML file: {html_file}")  # Debug log: HTML file initialized

# Fetch API data and parse resources
resources = fetch_api_data(api_url, relationship_type, related_item_type)

# Visualize the network graph
visualize_resource_network(resources, html_file)

# Close the HTML file after writing all content
with open(html_file, 'a') as f:
    f.write("</body></html>")
    print(f"Closed HTML file: {html_file}")  # Debug log: HTML file closed

print(f"Graphs saved in {html_file}")

Initialized HTML file: related_resource_network_graph.html
Fetching data from URL: https://api.datacite.org/dois?query=relatedIdentifiers.relationType:IsCollectedBy%20OR%20relatedIdentifiers.relationType:Collects
API response status code: 200
Parsed resources: [{'doi': '10.60922/vmsf-g485', 'resourceTypeGeneral': 'Collection', 'creators': [], 'contributors': [], 'relatedItems': []}, {'doi': '10.34973/tfwt-0x10', 'resourceTypeGeneral': 'Dataset', 'creators': [], 'contributors': [], 'relatedItems': [{'identifier': '10.1016/j.pec.2010.04.034', 'relationType': 'IsCollectedBy', 'resourceTypeGeneral': 'Instrument'}]}, {'doi': '10.34973/983b-a047', 'resourceTypeGeneral': 'Dataset', 'creators': [], 'contributors': [], 'relatedItems': [{'identifier': '10.1038/nn.4150', 'relationType': 'IsCollectedBy', 'resourceTypeGeneral': 'Instrument'}]}, {'doi': '10.34973/e8v8-rd88', 'resourceTypeGeneral': 'Dataset', 'creators': [], 'contributors': [], 'relatedItems': [{'identifier': '10.1038/nn.4150', 'rela

Displayed network graph for related identifier: 10.1016/j.pec.2010.04.034
Appended network graph to HTML file: related_resource_network_graph.html


Displayed network graph for related identifier: 10.1038/nn.4150
Appended network graph to HTML file: related_resource_network_graph.html


Displayed network graph for related identifier: https://www.ibm.com/products/spss-statistics
Appended network graph to HTML file: related_resource_network_graph.html


Displayed network graph for related identifier: https://atlasti.com/
Appended network graph to HTML file: related_resource_network_graph.html


...

Displayed network graph for related identifier: https://github.com/aravindhnivas/FELion_GUI3
Appended network graph to HTML file: related_resource_network_graph.html


Displayed network graph for related identifier: https://www.python.org/
Appended network graph to HTML file: related_resource_network_graph.html


Displayed network graph for related identifier: https://github.com/kalimu/genderizeR
Appended network graph to HTML file: related_resource_network_graph.html


Displayed network graph for related identifier: https://github.com/Darshan-Ramesh/EmpRecognition.git
Appended network graph to HTML file: related_resource_network_graph.html


Displayed network graph for related identifier: https://arxiv.org/pdf/2008.09470.pdf
Appended network graph to HTML file: related_resource_network_graph.html
Closed HTML file: related_resource_network_graph.html
Graphs saved in related_resource_network_graph.html


#README: Resource Relationship Network Visualization with Pyvis


# DataCite Relationship Visualization with PyVis

This project demonstrates how to fetch DOI relationship data from the DataCite API and visualize those relationships using the PyVis library in an interactive HTML graph.

## Table of Contents
	1.	Overview
	2.	Features
	3.	Installation
	4.	Usage
	5.	Code Structure
	6.	Customization
	7.	Known Limitations & Future Improvements
	8.	Contributing
	9.	License

## Overview

DataCite provides an API that returns metadata about DOIs (Digital Object Identifiers). This script specifically focuses on:
* Making paginated requests to the DataCite API to get records of resources (e.g., datasets, instruments, software).
* Filtering relationships (e.g., “IsCollectedBy”, “Collects”) to understand how different DOIs are interconnected.
* Generating an interactive graph using the PyVis library and displaying it in a browser or notebook environment (e.g., Jupyter, Google Colab).

By default, this script demonstrates how to:
* Identify all DOIs related by IsCollectedBy or Collects.
* Further filter the type of related item (e.g., “Instrument”).
* Display the final relationship graph in an HTML file.

## Features
* Pagination Handling: Automatically fetches all pages of results from the DataCite API.
* Flexible Relationship Filtering: Allows you to specify one or more relationship types (comma-separated).
* Optional Resource Type Filtering: Lets you target a specific related item type (e.g., “Instrument”).
* Interactive Graph Visualization: Uses PyVis to produce a dynamic, zoomable graph with tooltips and highlighting.

## Installation

1.	Clone or Download this repository (or simply copy the Python script into your local environment).
2.	Create and activate a Python virtual environment (optional):

```
python -m venv venv
source venv/bin/activate  # On Windows: .\venv\Scripts\activate

```

3.	Install required packages:

```
pip install -r requirements.txt
```

If you do not have a requirements.txt, you can manually install the key libraries:

```
pip install pyvis requests
```

(The script also uses IPython for inline display. If you are in Jupyter/Colab, you likely already have it.)

## Usage

1.	Open a Jupyter notebook (or Google Colab) and place the script’s code into a cell, or run it as a standalone .py file in an environment that can display or open HTML outputs.

2.	Configure the script parameters near the bottom:

```
relationship_type = "IsCollectedBy,Collects"   # Relationship types to look for
related_item_type = "Instrument"              # Related item type (optional)

api_url = (
    "https://api.datacite.org/dois?"
    "query=relatedIdentifiers.relationType:IsCollectedBy%20OR%20"
    "relatedIdentifiers.relationType:Collects"
)

html_file = "related_resource_network_graph_pyvis.html"
```

3.	Run the script. It will:
* Fetch the data from the specified API endpoint.
* Build a data structure of DOIs and their related items.
* Generate an HTML file (by default, related_resource_network_graph_pyvis.html) displaying the network graph.



4.	View the Graph:
* If using Jupyter or Colab, the script automatically displays the graph in an output cell.
* Otherwise, open the generated HTML file in your browser to explore the interactive graph.

## Code Structure
1.	Imports:

```
from IPython.core.display import display, HTML
from pyvis.network import Network
import requests
```

These are standard libraries plus PyVis for visualization.

2.	Custom Color Palette:

custom_colors provides an example color palette for styling nodes.

3.	fetch_api_data(url, relationship_type, related_item_type=None)

* Fetches pages of results from DataCite’s API.
* Accumulates data in a list.
* Uses parse_resources to process and filter each page’s results.

4.	parse_resources(data, relationship_type, related_item_type=None)

* Iterates over each item in the DataCite response.

* Extracts the DOI and resourceTypeGeneral.

* Filters related items based on relationship_type and (optionally) related_item_type.
* 	Returns a list of resource dictionaries, each containing:

```
{
  "doi": "10.1234/abc",
  "resourceTypeGeneral": "Dataset",
  "relatedItems": [
    {
      "identifier": "10.5678/xyz",
      "relationType": "IsCollectedBy",
      "resourceTypeGeneral": "Instrument"
    },
    ...
  ]
}
```

5.	visualize_resource_network_pyvis(resources, html_file)
* Creates a Network object (directed graph) with specific physics options for layout.
* For each resource:
	*	Adds a node for the central resource (the DOI).
	*	Adds nodes and edges for any related items.
	*	Exports the final graph to html_file.
	*	Displays the HTML file if running in a Jupyter-like environment.
6.	Script Execution
*	The final lines set the relationship_type, related_item_type, api_url, and html_file.
	•	Calls fetch_api_data and then visualize_resource_network_pyvis.

## Customization

* Relationship Types:
You can specify different relationship types (e.g., IsReferencedBy,References, IsPartOf,HasPart, etc.) by adjusting relationship_type.

Note: The script currently checks with a substring approach. If you require exact matching for multiple relationship types, you may need to split them (e.g., by commas) and compare individually.

* Related Item Type:
If you want to include all types of related items, set related_item_type = None.
*	Graph Appearance:
	*	Change colors in the custom_colors list.
	*	Modify physics options (e.g., springLength, centralGravity) by editing net.set_options(...).

*	Deduplication:
If the same DOI appears on multiple pages, you could create a single dictionary structure in the fetch_api_data function to avoid duplicates across pages.

## Known Limitations & Future Improvements
1.	Large Results:
If the results set is very large, you may run into performance issues with the HTML-based graph. Consider additional filtering or pagination logic.

2.	Relationship Matching:
The code uses a substring match (in or relation_type.lower() in relationship_type.lower()).
If you need more precise matching, split the relationship_type string by commas, trim whitespace, and match exactly.

3.	Error Handling:
Currently, the script stops if an HTTP error is encountered (response.raise_for_status()).
If partial data is acceptable, you may need to wrap calls in try/except blocks.

4.	Additional Metadata:
You may add or display more information about each node in the PyVis graph by modifying the labels or adding hover tooltips.

## Contributing

Contributions are welcome! If you find a bug or have a feature request, feel free to open an issue or submit a pull request. Please be clear about the changes you propose and ensure backward compatibility when possible.



In [3]:
!pip install pyvis

from IPython.core.display import display, HTML
from pyvis.network import Network
import requests
import json

# Custom color palette
custom_colors = ['#243B54', '#00B1E2', '#5B88B9', '#46BCAB', '#90D7CD', '#BC2B66']

def fetch_api_data(url, relationship_type, related_item_type=None):
    """
    Fetch data from the API endpoint with dynamic relationship type and related item type.
    Paginates through 'links.next' if available.
    """
    resources = []
    while url:
        response = requests.get(url)
        print(f"Fetching data from URL: {url}")  # Debug log
        response.raise_for_status()  # Raise an error if request was unsuccessful
        data = response.json()
        print(f"API response status code: {response.status_code}")  # Debug log

        # Parse resources from this page of results
        page_resources = parse_resources(data.get('data', []), relationship_type, related_item_type)
        resources.extend(page_resources)

        # Go to the next page if available
        url = data.get('links', {}).get('next')
    return resources

def parse_resources(data, relationship_type, related_item_type=None):
    """
    Parse a list of items (from one page of the API) and extract DOIs + related items
    matching the given relationship_type and optional related_item_type.
    Returns a list of unique resources (by DOI) for that page.
    """
    resources_dict = {}

    for item in data:
        # Normalize resourceTypeGeneral to handle variations like hyphens and casing.
        # (Currently not used further below, but could be used if you want the normalized value.)
        resource_type_normalized = (item['attributes']['types']
                                    .get('resourceTypeGeneral', '')
                                    .replace('-', '')
                                    .lower())

        doi = item['attributes'].get('doi', 'No DOI available')

        # Initialize resource if we haven't seen this DOI yet
        if doi not in resources_dict:
            resources_dict[doi] = {
                "doi": doi,
                "resourceTypeGeneral": item['attributes']['types']
                                            .get('resourceTypeGeneral',
                                                 'No resourceTypeGeneral available'),
                "relatedItems": []
            }

        # If 'relatedIdentifiers' exist, filter them by relationship_type and resource_type
        if 'relatedIdentifiers' in item['attributes']:
            for related in item['attributes']['relatedIdentifiers']:
                relation_type = related.get('relationType', '')
                resource_type_related = related.get('resourceTypeGeneral', '')

                # Check if this relation matches the desired relationship type
                # e.g. "IsCollectedBy,Collects" -> we do a substring check below
                if (relation_type and relationship_type and
                        relation_type.lower() in relationship_type.lower()):

                    # If related_item_type is None, accept all;
                    # otherwise compare normalized resource types
                    if (related_item_type is None or
                       resource_type_related.replace('-', '').lower() ==
                       related_item_type.replace('-', '').lower()):

                        resources_dict[doi]['relatedItems'].append({
                            "identifier": related.get('relatedIdentifier',
                                                      'No identifier available'),
                            "relationType": relation_type,
                            "resourceTypeGeneral": resource_type_related
                        })

    print(f"Parsed resources: {resources_dict}")  # Debug log
    return list(resources_dict.values())

def visualize_resource_network_pyvis(resources, html_file):
    """
    Visualize the resource network graph using pyvis and display the
    generated HTML file inline (e.g., in a Jupyter/Colab environment).
    """
    net = Network(notebook=True, directed=True)
    net.set_options("""
        var options = {
          "physics": {
            "forceAtlas2Based": {
              "gravitationalConstant": -50,
              "centralGravity": 0.01,
              "springLength": 230,
              "springConstant": 0.08
            },
            "maxVelocity": 146,
            "solver": "forceAtlas2Based",
            "timestep": 0.35,
            "stabilization": {
              "enabled": true,
              "iterations": 200
            }
          }
        }
    """)

    for resource in resources:
        # Add the main resource (DOI) as a node
        resource_label = f"{resource['resourceTypeGeneral']}: DOI: {resource['doi']}"
        net.add_node(resource['doi'], label=resource_label, color=custom_colors[0])

        # Add each related item as a node and connect with an edge
        for related_item in resource["relatedItems"]:
            related_label = (f"{related_item['resourceTypeGeneral']}: "
                             f"{related_item['identifier']}")
            net.add_node(related_item['identifier'],
                         label=related_label,
                         color=custom_colors[1])
            net.add_edge(resource['doi'],
                         related_item['identifier'],
                         title=related_item['relationType'])

    # Generate and save the interactive network to HTML
    net.show(html_file)

    # Display the generated HTML inline (works in Jupyter/Colab)
    with open(html_file, 'r') as f:
        html_content = f.read()
        display(HTML(html_content))

    print(f"Visualized network graph saved and displayed from: {html_file}")

# Example usage
relationship_type = "IsCollectedBy,Collects"  # Comma-separated for substring matching
related_item_type = "Instrument"

api_url = (
    "https://api.datacite.org/dois?"
    "query=relatedIdentifiers.relationType:IsCollectedBy%20OR%20"
    "relatedIdentifiers.relationType:Collects"
)

html_file = "related_resource_network_graph_pyvis.html"

# Fetch API data
resources = fetch_api_data(api_url, relationship_type, related_item_type)

# Visualize with pyvis
visualize_resource_network_pyvis(resources, html_file)
